In [2]:
# Les imports sont préparé ici
# n'enlevez pas les % car il permettent le reload de modules ou l'affichage dans le notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import random
import pickle as pkl
import re
import matplotlib.pyplot as plt
import platform
from sklearn.metrics import f1_score
import spacy
from spacy import displacy

In [16]:
#reading files
#Méthode qui renvoie un tableau contenant les lignes des consensus uniquement 
def readingfile(file):
    if platform.system() == 'Windows':
        f1 = open(file, "r", encoding = "UTF-8")
    else:
        f1 = open(file, "r")

    data_ = []
    data_cons = []

    rows = f1.readlines()
    for line in rows:   #on itère sur les lignes

            #line.decode("utf-8")
        en_tete=line.split(')')[0]                                      #On récupère seulement l'en_tete de chaque ligne
        if "consensus" in en_tete:                                        #On vérifie que cet entete contient le consensus
            data_.append(line.split(')')[1])                                          #si on a consensus dans la ligne courante on l'ajoute au tableau de renvoie
            data_cons.append(line)
            
    # closing files
    f1.close()

    return data_, data_cons


In [27]:
file = "train_label_temp3.txt"
data, datacons = readingfile(file)


In [39]:
""" Cleaning Tweets """
datacons = datacons.str.lower()

# removing special characters and numbers
datacons = datacons.apply(lambda x : re.sub("[^a-z\s]","",x) )

# removing stopwords
from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))
datacons = datacons.apply(lambda x : " ".join(word for word in x.split() if word not in stopwords ))

data.head.tolist()


[autoreload of srsly.ruamel_yaml.util failed: Traceback (most recent call last):
  File "C:\Users\aline\miniconda3\envs\env_pytorch\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\aline\miniconda3\envs\env_pytorch\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\aline\miniconda3\envs\env_pytorch\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "C:\Users\aline\miniconda3\envs\env_pytorch\lib\site-packages\IPython\extensions\autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "C:\Users\aline\miniconda3\envs\env_pytorch\lib\site-packages\IPython\extensions\autoreload.py", line 280, in update_instances
    ref.__class__ = new
  File "C:\Users\aline\miniconda3\envs\env_pytorch\lib\site-packages\srsly\ruamel_yaml\util.py", line 49, in __setatt

AttributeError: 'list' object has no attribute 'str'

In [38]:
nlp_en = spacy.load("en_core_web_sm")
#nlp_fr = spacy.load("fr_core_news_sm")

list_doc = []

for i in data :
    doc = nlp_en(i)
    list_doc.append(doc)

    
c_doc = Doc.from_docs(list_doc)
    
    
displacy.render(c_doc,style = 'ent')


In [47]:
#Nettoyage des données --> on enlève les mots iil

from spacy.lang.en.stop_words import STOP_WORDS

filtered_sent = []

document = nlp_en(c_doc)

for doc in list_doc :
    for word in doc :
        if word.is_stop == False:
            filtered_sent.append(word)
print("Filtered Sentence:",filtered_sent)

Filtered Sentence: [ , London, #, Microsoft, Partner, Business, Briefing, tomorrow, -, :,  , Wow!RT, @fmlopez48, #, Microsoft, trabaja, en, tecnología, con, la, que, cualquier, superficie, podrá, ser, usada, como, pantalla, táctil, http://t.co/1qbeGAI5, 
,  , Etre, influent, sur, #, Twitter, :, publication, de, contenus, qui, engagent, à, l'action, ., http://t.co/H2qmDwpP, EN, @jeffbullas, 
,  , Great, Mission, :, :, Ice, Cream, Sandwich, ,, #, Google, mission, build, mobile, OS, works, phones, tablets, 
,  , issues, #, Siri, reaching, network, today, ., uses, signal, ?, @apple, 
,  , indiretaa, ta, solta, hoje, nesse, #, twitter, UAHSUAHSUASA, ', E, PRA, ELA, S2, 
,  , Bummer, !, @Yahoo, !, iPad, fantasy, football, app, keeps, crashing, ;, sure, app, new, @apple, iOS, 5, ., apps, running, ., 
,  , @siobhancoda, anti, virus, 90, day, trial, ,, leave, exposed, near, future, #, free, #, microsoft, #, antivirus, #, software, 
,  , RT, @EA_Jorge, :, day, 80, -, Xbox, LIVE, acct, hacked, .,

In [13]:
for token in c_doc:
    print("{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}\t{8}".format(
        token.text,
        token.idx,
        token.lemma_,
        token.is_punct,
        token.is_space,
        token.shape_,
        token.pos_,
        token.tag_,
        token.ent_type_
    ))

 	0	 	False	True	 	SPACE	_SP	
#	1	#	True	False	#	SYM	$	
Microsoft	2	Microsoft	False	False	Xxxxx	PROPN	NNP	ORG
lanza	12	lanza	False	False	xxxx	ADJ	JJ	
sitio	18	sitio	False	False	xxxx	NOUN	NN	
oficial	24	oficial	False	False	xxxx	ADJ	JJ	
de	32	de	False	False	xx	ADP	IN	
#	35	#	True	False	#	X	FW	
Skype	36	Skype	False	False	Xxxxx	PROPN	NNP	
http://t.co/fWDALlIe	42	http://t.co/fwdallie	False	False	xxxx://x.xx/xXXXXxXx	NUM	CD	
